In [ ]:
%pip install pandas datasets openai python-dotenv tqdm timeout-decorator fuzzysearch sqlalchemy regex sentence-transformers tabulate

In [ ]:
import pandas as pd
from datasets import load_dataset, Dataset
import openai
import os
from dotenv import load_dotenv
from typing import Callable, List, Optional
from tqdm import tqdm
import timeout_decorator
import regex
import time
from sqlalchemy import create_engine, text
import sqlite3
import threading
from typing import List, Tuple, Dict
import numpy as np
from fuzzysearch import find_near_matches
from functools import lru_cache
import re
import time
from sqlalchemy import create_engine, MetaData, Table, text, types
from sqlalchemy.orm import sessionmaker

### Loading API keys

In [3]:
load_dotenv();

### Utils

In [4]:
def replace_emojis_with_unique_symbols(df):
    emoji_pattern = regex.compile(r'\p{So}|\p{Cn}')
    
    def replace_emojis(column_name):
        # Find all emojis using regex
        emojis = regex.findall(r'\X', column_name)  # Match grapheme clusters
        replacements = {}
        for emoji in emojis:
            # Check if it's an emoji (Unicode "Other Symbols" or related categories)
            if emoji_pattern.match(emoji):
                # Replace emoji with a unique symbol based on its Unicode representation
                unique_symbol = f"_IMG{'_'.join(f'{ord(c):X}' for c in emoji)}_"
                replacements[emoji] = unique_symbol
        
        # Replace all emojis in one go
        for emoji, unique_symbol in replacements.items():
            column_name = column_name.replace(emoji, unique_symbol)
        return column_name

    # Apply the replacement function to all column names
    df.columns = df.columns.map(replace_emojis)
    return df


In [5]:
# Initialize a dictionary to cache preloaded datasets
preloaded_datasets = {}

def load_data(name: str = "semeval", split: str = "dev") -> Dataset:
    """Load dataset from local parquet file or download if not available."""
    local_path = f"./{name}_{split}.parquet"
    
    if os.path.exists(local_path):
        return Dataset.from_parquet(local_path)
    
    dataset = load_dataset("cardiffnlp/databench", name=name, split=split)
    dataset.to_parquet(local_path)
    return dataset

def load_table(dataset_name: str, is_sample: bool = False) -> pd.DataFrame:
    """Load table from local parquet file or remote source if not cached."""
    file_type = "sample" if is_sample else "all"
    key = f"{dataset_name}_{file_type}"
    local_path = f"./{dataset_name}_{file_type}.parquet"
    
    if key not in preloaded_datasets:
        if os.path.exists(local_path):
            df = pd.read_parquet(local_path)
        else:
            df = pd.read_parquet(f"hf://datasets/cardiffnlp/databench/data/{dataset_name}/{file_type}.parquet")
            
        df.columns = df.columns.str.replace(r'<gx:.*?>', '', regex=True)
        df = replace_emojis_with_unique_symbols(df)
        # df.columns = [f"{col}_{i}" for i, col in enumerate(df.columns)]
        df.columns = [f"{col}" for col in df.columns]
        df._processed = True
        preloaded_datasets[key] = df
    else:
        df = preloaded_datasets[key]
        
    return df

def load_sample(name: str = "qa") -> pd.DataFrame:
    """Load sample data from local parquet file or remote source if not available."""
    local_path = f"./{name}_sample.parquet"
    
    if os.path.exists(local_path):
        return pd.read_parquet(local_path)
    
    return pd.read_parquet(f"hf://datasets/cardiffnlp/databench/data/{name}/sample.parquet")

In [24]:
class OpenAIClient:
    def __init__(self):
        load_dotenv()
        self.client = openai.OpenAI(
            api_key=os.getenv('OPENROUTER_API_KEY2'),
            base_url="https://openrouter.ai/api/v1"
        )

    def generate_response(self, prompt: str, max_tokens=3000, retries=3, model="meta-llama/llama-3.3-70b-instruct") -> str:
        sleep_durations = [15, 30, 60]  # Sleep durations in seconds

        for attempt in range(retries):
            try:

                response = self.client.chat.completions.create(
                    model=model,
                    messages=[
                        {"role": "system", "content": "You are a best in class instruction following assistant. You excel in tasks with data. You reason very thorougly and step-by-step."},
                        {"role": "user", "content": prompt}
                    ],
                    temperature=0,
                    max_tokens=max_tokens
                )
                # print(response)

                if response.choices:
                    content = response.choices[0].message.content.strip()
                    # print(f"{model}: {content}")
                    return content
                else:
                    raise ValueError("No response choices available.")

            except Exception as e:
                print(f"Attempt {attempt + 1} failed with error: {str(e)}")
                if attempt < len(sleep_durations):
                    print(f"Sleeping for {sleep_durations[attempt]} seconds before retrying...")
                    time.sleep(sleep_durations[attempt])
                else:
                    return f"__CODE_GEN_ERROR__: {str(e)}"


In [25]:
def get_relevant_columns_only(df, question, ai_client):
     #Get relevant columns
    prompt = f"Given the question: '{question}', which columns from the dataset are necessary to answer it? The dataset contains the following columns: {', '.join(df.columns)}. Please provide extensive reasoning and only then the column names as a comma-separated list started with [ and ended with ]."
    # print(prompt)
    columns_text = ai_client.generate_response(prompt, model="meta-llama/llama-3.2-3b-instruct").strip()
    # print(f"Columns identified: {columns_text}")

    columns_list = [col.strip().strip("'\"`") for col in columns_text[columns_text.rfind('[')+1:columns_text.rfind(']')].split(',')]
    # print(f"Columns identified2: {columns_list}")

    return df[columns_list]

In [26]:
import numpy as np
from sentence_transformers import SentenceTransformer
import torch

# Initialize model globally to avoid reloading
_model = None

def get_model():
    global _model
    if _model is None:
        _model = SentenceTransformer('all-MiniLM-L6-v2')
    return _model

def get_relevant_rows_by_cosine_similarity(df, question, ai_client):
    # Use GPU if available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = get_model().to(device)

    # Get relevant columns efficiently without copy
    df = get_relevant_columns_only(df, question, ai_client=ai_client)
    
    # Vectorized string concatenation
    text_data = df.astype(str).values.sum(axis=1)
    
    try:
        # Sequential encoding
        embeddings = model.encode(
            text_data,
            batch_size=256,
            show_progress_bar=False,
            convert_to_tensor=True,
            device=device
        )
        question_embedding = model.encode(
            [question],
            show_progress_bar=False,
            convert_to_tensor=True,
            device=device
        )[0]

        # Vectorized similarity calculation
        embeddings = embeddings.cpu().numpy()
        question_embedding = question_embedding.cpu().numpy()
        
        # Fast cosine similarity using matrix operations
        similarities = np.dot(embeddings, question_embedding)
        norms = np.linalg.norm(embeddings, axis=1) * np.linalg.norm(question_embedding)
        similarities = np.divide(similarities, norms, where=norms!=0)
        
        # Fast filtering using numpy operations
        top_indices = np.argpartition(similarities, -10)[-10:]
        return df.iloc[top_indices]
        
    except Exception as e:
        print(f"Error in similarity calculation: {str(e)}")
        return df.head(3)

---

### Evaluation

In [27]:
def evaluate_responses(responses: List[str], dataset, is_sample: bool = True) -> Tuple[float, List]:
    start_time = time.time()
    correct = 0
    truths = dataset["sample_answer" if is_sample else "answer"]
    logs = []
    
    if not responses or not truths:
        print("Responses or truths are empty, skipping evaluation.")
        return (0.0, logs)
    
    for response, truth, semantic, question in zip(responses, truths, dataset["type"], dataset["question"]):
        is_correct = compare_results(response, truth, semantic)
        
        if is_correct:
            correct += 1
        else:
            print(response, " || " , truth, "||", question, f"verdict={is_correct}" )
        logs.append((response, truth, semantic, question, is_correct))
    
    if not logs:
        print("No logs were generated during evaluation.")
    
    return (correct / len(truths), logs)

def compare_results(value, truth, semantic):
    STRIP_CHARS = "[]'\" "
    semantic = semantic.strip()
    result = False
    
    if semantic == "boolean":
        result = str(value).strip(STRIP_CHARS).lower() == str(truth).strip(STRIP_CHARS).lower()
    elif semantic == "category":
        if value is None and truth is None:
            result = True
        elif value is None or truth is None:
            result = False
        else:
            value_str = str(value).strip(STRIP_CHARS)
            truth_str = str(truth).strip(STRIP_CHARS)
            if value_str == truth_str:
                result = True
            else:
                try:
                    value_date = pd.to_datetime(value_str).date()
                    truth_date = pd.to_datetime(truth_str).date()
                    result = value_date == truth_date
                except (ValueError, TypeError):
                    if not value_str and not truth_str:
                        result = True
                    else:
                        result = value_str == truth_str
    elif semantic == "number":
        try:
            value_cleaned = ''.join(char for char in str(value) if char.isdigit() or char in ['.', '-'])
            truth_cleaned = ''.join(char for char in str(truth) if char.isdigit() or char in ['.', '-'])
            result = round(float(value_cleaned), 2) == round(float(truth_cleaned), 2)
        except:
            result = False
    elif semantic == "list[category]":
        try:
            value_list = [item.strip(STRIP_CHARS) for item in str(value).strip('[]').split(',')]
            truth_list = [item.strip(STRIP_CHARS) for item in str(truth).strip('[]').split(',')]
            if len(value_list) != len(truth_list):
                result = False
            else:
                try:
                    value_dates = [pd.to_datetime(item).date() for item in value_list]
                    truth_dates = [pd.to_datetime(item).date() for item in truth_list]
                    result = set(value_dates) == set(truth_dates)
                except (ValueError, TypeError):
                    result = set(value_list) == set(truth_list)
        except Exception as exc:
            result = False
    elif semantic == "list[number]":
        try:
            value_list = sorted(round(float(''.join(c for c in v.strip() if c.isdigit() or c in ['.', '-'])), 2) for v in str(value).strip('[]').split(',') if v.strip())
            truth_list = sorted(round(float(''.join(c for c in t.strip() if c.isdigit() or c in ['.', '-'])), 2) for t in str(truth).strip('[]').split(',') if t.strip())
            
            if len(value_list) != len(truth_list):
                result = False
            else:
                result = set(value_list) == set(truth_list)
        except Exception as exc:
            result = False
    else:
        raise ValueError(f"Unsupported semantic type: {semantic}. Please use one of the following: 'text', 'number', 'list[category]', 'list[number]'.")
    
    return result


---

### SQL/Pandas Execution

In [28]:
def try_python_execution(code: str, dataset_name: str, is_sample: bool, timeout: int = 5) -> Tuple[str, bool]:
    def execute_with_timeout(code, local_vars, timeout):
        exception = [None]
        
        def target():
            try:
                # Ensure pandas and numpy are available in the execution context
                exec(f"import numpy as np; import pandas as pd; {code}", {}, local_vars)
                local_vars['success'] = True
            except Exception as e:
                local_vars['result'] = str(e)
                local_vars['success'] = False

        thread = threading.Thread(target=target)
        thread.start()
        thread.join(timeout)
        if thread.is_alive():
            return "Execution timed out", False
        if exception[0]:
            return str(exception[0]), False
        return local_vars.get('result', "Unknown error"), local_vars.get('success', False)

    # Load the dataset and prepare local variables
    df = load_table(dataset_name, is_sample)
    local_vars = {'df': df, 'pd': pd, 'np': np}
    
    # Execute the code with a timeout
    return execute_with_timeout(code, local_vars, timeout)

---

### Preparing prompts


In [29]:
def generate_prompt(row: dict, is_sample) -> str:
    """
    Generate prompt for the LLM.
    """
    start_time = time.time()
    dataset = row["dataset"]
    question = row["question"]
    df = load_table(dataset, is_sample)

    exp_prompt2 = f""""
    
1. You are two of the most esteemed Pandas DataScientists engaged in a heated and truth-seeking debate. You are presented with a dataframe and a question. Begin dialogue by rigorously discussing your reasoning step by step, ensuring to address all aspects of the checklist. In your discourse, meticulously articulate the variable type necessary to derive the answer and confirm that each column referenced is indeed present in the dataframe. Conclude your debate by providing the code to answer the question, ensuring that the variable result is explicitly assigned to the answer. Remember, all code must be presented in a single line, with statements separated by semicolons. 
2. Refrain from importing any additional libraries beyond pandas and numpy.
3. The dataframe, df, is already populated with data for your analysis; do not initialize it, but focus solely on manipulating df to arrive at the answer.
4. If the question requires multiple entries, always utilize .tolist() to present the results.
5. If the question seeks a single entry, ensure that only one value is output, even if multiple entries meet the criteria.

You MUST FOLLOW THE CHECKLIST, ANSWER EACH OF ITS QUESTIONS (REASONING STEP), AND ONLY THEN OUTPUT THE FINAL ANSWER BASED ON THOSE ANSWERS:
1) How many values should be in the output?
2) Values (or one value) from which column (only one!) should the answer consist of?
3) What should be the type of value in the answer? 

Example of a task:
Question: Identify the top 3 departments with the most employees. 
<Columns> = ['department', 'employee_id'] 
<First_row> = ('department': 'HR', 'employee_id': 101)
Reasoning: Count the number of employees in each department, sort, and get the top 3. The result should be a list of department names. 
Checklist: 
1) The output should consist of 3 values.
2) The values should come from the 'department' column.
3) The type of value in the answer should be a list of strings.
Code: result = df['department'].value_counts().nlargest(3).index.tolist()

Your data to process:
<question> = {question}

- Make absolute sure that all columns used in query are present in the table.
<columns_in_the_table> = {[col for col in df.columns]}
<first_rows_of_table> = {df.head(3).to_string()}
YOUR Reasoning through dialogue and Code (Start final code part by "Code:"):    

"""

    experimental_prompt = f""" 
1. You are a best in the field Pandas DataScientist. You are given a dataframe and a question. You should spell out your reasoning step by step and only then provide code to answer the question. In the reasoning state it is essentianl to spell out the answers' variable type that should be sufficient to answer the question. Also spell out that each column used is indeed presented in the table. In the end of your code the variable result must be assigned to the answer to the question. One trick: all code should be in one line separated by ; (semi-columns) but it is no problem for you. 
2. Avoid importing any additional libraries than pandas and numpy.
3. All data is already loaded into df dataframe for you, you MUST NOT initialise it, rather present only manipulations on df to calculate the answer. 
4. If the question ask for several entries alsways use .tolist().
5. If the question ask for one entry, make sure to output only one, even if multiple qualify.

You MUST FOLLOW THE CHECKLIST, ANSWER EACH OF ITS QUESTION (REASONING STEP) AND ONLY THEN OUTPUT THE FINAL ANSWER BASED ON THOSE ANSWERS:
1) How many values should be in the output?
2) Values (or one value) from which column (only one!) should the answer consist of?
3) What is should be the type of value in the answer? 

Example:
Question: What are the top 5 cities with the highest average temperature? 
<Columns> = ['city', 'temperature'] 
<First_row> = ('city': 'New York', 'temperature': 85)
Reasoning: Calculate the average temperature for each city, sort them, and select the top 5. The result should be a list of city names. 
Checklist: 
1) The output should consist of 5 values.
2) The values should come from the 'city' column.
3) The type of value in the answer should be a list of strings.
Code: result = df.groupby('city')['temperature'].mean().nlargest(5).index.tolist()

Example:
Question: Which product has the highest sales volume? 
<Columns> = ['product', 'sales'] 
<First_row> = ('product': 'Laptop', 'sales': 150)
Reasoning: Sum the sales volume for each product and find the one with the maximum total. The result should be a single product name as a string. 
Checklist: 
1) The output should consist of 1 value.
2) The value should come from the 'product' column.
3) The type of value in the answer should be a string.
Code: sales_volume = df.groupby('product')['sales'].sum(); result = sales_volume.idxmax()

Example:
Question: Identify the top 3 departments with the most employees. 
<Columns> = ['department', 'employee_id'] 
<First_row> = ('department': 'HR', 'employee_id': 101)
Reasoning: Count the number of employees in each department, sort, and get the top 3. The result should be a list of department names. 
Checklist: 
1) The output should consist of 3 values.
2) The values should come from the 'department' column.
3) The type of value in the answer should be a list of strings.
Code: result = df['department'].value_counts().nlargest(3).index.tolist()

Your data to process:
<question> = {question}

- Make absolute sure that all columns used in query are present in the table.
<columns_in_the_table> = {[col for col in df.columns]}
<first_rows_of_table> = {df.head(3).to_markdown()}
YOUR Reasoning and Code (Start code part by "Code:"):
"""
    # print([col for col in df.columns])
    return exp_prompt2

In [30]:
def create_reflection_prompt(question: str, df: pd.DataFrame, tracebacks: List[str]) -> str:
    return (
        "The following solutions failed for the task: \"{question}\"\n\n"
        + '\n'.join([f'Solution {i+1} Error:\n{traceback}\n' for i, traceback in enumerate(tracebacks)])
        + "\nDF info: \n"
        + "<columns_to_use> = " + str([(col, str(df[col].dtype)) for col in df.columns]) + "\n"
        + "<first_row_of_table> = " + str(df.head(1).to_dict(orient='records')[0]) + "\n"
        + "YOUR answer in a single line of pandas code:\n"
        + "Please craft a new solution considering these tracebacks. Output only fixed solution in one line:\n"
    )

In [31]:
def generate_voting_prompt(solutions: List[Dict], question: str, dataset_name: str, is_sample: bool):
    df = load_table(dataset_name, is_sample)

    voting_prompt = f"""
Examples of deducing answer types:  
1. If the question is "Do we have respondents who have shifted their voting preference?" the answer type is **Boolean** because the response should be True/False.  
2. If the question is "How many respondents participated in the survey?" the answer type is **Integer**
3. If the question is "List the respondents who preferred candidate X?" the answer type is **List** because the response requires a collection of values.  
4. If the question is "What is the average age of respondents?" the answer type is **Number** because the response should be a decimal value.  
5. If the question is "What is the name of the candidate with the highest votes?" the answer type is **String** because the response is a single textual value.  

Given the following solutions and their results for the task: "{question}"  

{'\n'.join([f'Solution Number {i+1}:\nCode: {r["code"]} Answer: {str(r["result"])[:50]} (may be truncated)\n' for i, r in enumerate(solutions)])}  

Instructions:  
- Deduce the most probable and logical result to answer the given question. Then output the number of the chosen answer.
- If you are presented with end-to-end solution, it should not be trusted for numerical questions, but it is okay for other questions.
- Make absolute sure that all columns used in solutions are present in the table. SQL query may use additional double quotes around column names, it's okay, always put them. Real Tables columns are: {df.columns}
- If the column name contain emoji or unicode character make sure to also include it in the column names in the query.
- If several solutions are correct, return the lowest number of the correct solution.  
- Otherwise, return the solution number that is most likely correct.
- If the question ask for one entry, make sure to output only one, even if multiple qualify.

You should spell out your reasoning step by step and only then provide code to answer the question. In the reasoning state it is essentianl to spell out the answers' variable type that should be sufficient to answer the question. Also spell out that each column used is indeed presented in the table. The most important part in your reasoning should be dedicated to comparing answers(results) from models and deducing which result is the most likely to be correct, then choose the model having this answer.
First, predict the answer type for the question. Then give your answer which is just number of correct answer with predicted variable type.  Start reasoning part with "REASONING:" and final answer with "ANSWER:".
"""
    return voting_prompt

In [32]:
def get_sql_prompt(failed_solutions, question, column_names, df, ai_client):
        return f"""
Some Python attempts failed with errors:
{', '.join([r["result"] for r in failed_solutions])}

The task was: {question}

Here are some examples of SQL queries for similar tasks:
Example 1:
Task: Is there any entry where age is greater than 30?
REASONING: 
1. Identify the column of interest, which is 'age'.
2. Determine the condition to check, which is 'age > 30'.
3. Use the SELECT statement to retrieve a boolean result indicating the presence of such entries.
4. Apply the WHERE clause to filter rows based on the condition 'age > 30'.
5. Use the EXISTS clause to ensure the query outputs 'True' if any row matches the condition, otherwise 'False'.
6. Ensure the table name is 'temp_table' and the column name is enclosed in double quotes to handle any spaces or special characters.
7. Verify that the query outputs 'True' or 'False' when presented with a yes or no question.
CODE: ```SELECT CASE WHEN EXISTS(SELECT 1 FROM temp_table WHERE "age" > 30) THEN 'True' ELSE 'False' END;```

Example 2:
Task: Count the number of entries with a salary above 50000.
REASONING: 
1. Identify the column of interest, which is 'salary'.
2. Determine the condition to filter the data, which is 'salary > 50000'.
3. Use the SELECT COUNT(*) statement to count the number of rows that meet the condition.
4. Apply the WHERE clause to filter rows based on the condition 'salary > 50000'.
5. Ensure the table name is 'temp_table' and the column name is enclosed in double quotes to handle any spaces or special characters.
CODE: ```SELECT COUNT(*) FROM temp_table WHERE [salary] > 50000;```

Write a correct fault-proof SQL SELECT query that solves this precise task.
Rules:
- Your SQL query should be simple with just SELECT statement, without WITH clauses.    
- Your SQL query should output the answer, without a need to make any intermediate calculations after its finish
- Use only basic SQL operations from SQLAlchemy (SELECT, FROM, WHERE, GROUP BY, etc.)
- Make sure not to use "TOP" operation as it is not presented in SQLite 
- If present with YES or NO question, Query MUST return 'True' or 'False'
- Write pure SQL only without any quotes around the query
- If the question asks about several values, your query should return a list
- If the question ask for one entry, make sure to output only one, even if multiple qualify.
- Equip each column name into double quotes
- Equip each string literal into double quotes
- Use COALESCE( ..., 0) to answer with 0 if no rows are found and the question asks for the number of something.
- If it is Yes/No question, make sure that your query output only True or False.
- In the reasoning spell out that each column used is indeed presented in the table.
- Enclose your code into ```
- SELECT close MUST contain ONLY ONE column. For example, it must be only author's name, not name and id.
- Before writing code give extensive yet precise and specific reasoning for each step of your solution. Start reasoning part by "REASONING:" and code part by "CODE:"

Table name is 'temp_table'.
Available columns and types: {', '.join([f"{col}: {str(type(df[col].iloc[0]))}" for col in column_names])}


Top 3 rows with highest cosine similarity: {get_relevant_rows_by_cosine_similarity(df, question, ai_client).head(3).to_markdown()}
YOUR RESPONSE:

"""

### Generating Solution

In [43]:
def get_python_solutions(question: str, dataset_name: str, is_sample: bool, ai_client, n_attempts: int = 1) -> List[Dict]:
    """
    Get multiple solutions from LLM and execute them.
    """
    solutions = []

    for i in range(n_attempts):
        model = select_model(i)

        start_time_prompt = time.time()
        prompt = generate_prompt({"question": question, "dataset": dataset_name}, is_sample)
        end_time_prompt = time.time()
        print(f"Time for generate_prompt: {end_time_prompt - start_time_prompt:.4f} seconds")

        start_time_response = time.time()
        code_response = ai_client.generate_response(prompt, model=model)
        # print(code_response)
        end_time_response = time.time()
        print(f"Time for generate_response: {end_time_response - start_time_response:.4f} seconds")

        start_time_clean = time.time()
        code_response = clean_code_response(code_response)
        end_time_clean = time.time()
        print(f"Time for clean_code_response: {end_time_clean - start_time_clean:.4f} seconds")

        try:
            start_time_execution = time.time()
            result, success = try_python_execution(code_response, dataset_name, is_sample)
            end_time_execution = time.time()
            print(f"Time for try_python_execution: {end_time_execution - start_time_execution:.4f} seconds")
        except Exception as e:
            result, success = None, False
            print(f"An error occurred during execution: {e}")

        result = format_result(result)
        solutions.append({"code": code_response, "result": result, "success": success})

    # log_solutions(question, dataset_name, solutions[-1])

    if sum(solution['success'] for solution in solutions) < 2:
        solutions.extend(handle_failed_solutions(question, dataset_name, is_sample, ai_client, solutions, n_attempts))

    return solutions

def select_model(attempt: int) -> str:
    models = ["mistralai/codestral-2501", "meta-llama/llama-3.3-70b-instruct", "qwen/qwen-2.5-coder-32b-instruct"]
    return models[attempt % len(models)]

def clean_code_response(code_response: str) -> str:
    
    matches = find_near_matches('Code:', code_response, max_l_dist=2)
    if matches:
        code_response = code_response[matches[-1].end:]
    else:
        raise ValueError("Expected 'Code:' in the response but not found.")
    
    # code_response = code_response.split("```", 1)[-1] if "```" in code_response else code_response
    code_response = code_response.replace("*", "").replace("`", "").replace("python", "").replace('return', '').strip()
    code_response = code_response.splitlines()[-1] if code_response.splitlines() else ""
    import re

    # Remove specific DataFrame creation pattern from code_response
    code_response = re.sub(r"df\s*=\s*pd\.DataFrame\(\[\{.*?\}\]\);", "", code_response, flags=re.DOTALL)
    return code_response

def format_result(result) -> str:
    if isinstance(result, str):
        result = result.replace('`', '').replace('"', '').replace("'", '')
    elif isinstance(result, list):
        result = str(result[0]) if len(result) == 1 else str(result)
    return result

def log_solutions(question: str, dataset_name: str, solutions: List[Dict]):
    with open("1_test.txt", "a", encoding='utf-8') as f:
        f.write(f"---\n{question}\n{load_table(dataset_name).columns}\n{solutions['code']}\n{solutions['result']}\n----")

def handle_failed_solutions(question: str, dataset_name: str, is_sample: bool, ai_client, solutions: List[Dict], n_attempts: int) -> List[Dict]:

    df = load_table(dataset_name, is_sample)
    tracebacks = [solution['result'] for solution in solutions if isinstance(solution['result'], str)]
    reflection_prompt = create_reflection_prompt(question, df, tracebacks)
    new_solutions = []
    for j in range(n_attempts):
        model = select_model(j)
        new_code_response = ai_client.generate_response(reflection_prompt, model=model)
        new_code_response = clean_code_response(new_code_response)
        result, success = try_python_execution(new_code_response, dataset_name, is_sample)
        new_solutions.append({"code": new_code_response, "result": result, "success": success})
    return new_solutions




## SQL code gen

In [34]:
from sqlalchemy import create_engine, types, event, text
from functools import lru_cache
from sqlalchemy.types import UnicodeText

def get_db_connection():
    """Cache single SQLite in-memory connection"""
    engine = create_engine('sqlite:///:memory:', 
                         echo=False
                         )
    return engine


In [35]:
def sql_fallback(dataset_name: str, question: str, is_sample: bool, failed_solutions: List[Dict], ai_client, n_tries: int = 1) -> List[Dict]:

    import time

    start_time = time.time()
    engine = get_db_connection()
    # print(f"get_db_connection took {time.time() - start_time:.4f} seconds")

    start_time = time.time()
    table_name = f"table_{dataset_name}_{(str(is_sample))}"  # Unique table name
    # print(f"Table name generation took {time.time() - start_time:.4f} seconds")

    start_time = time.time()
    df = load_table(dataset_name, is_sample)
    # print(f"load_table took {time.time() - start_time:.4f} seconds")

    start_time = time.time()
    df.to_sql(table_name, con=engine, index=False, if_exists='replace', dtype={'column_name': types.UnicodeText})
    # print(f"df.to_sql took {time.time() - start_time:.4f} seconds")

    start_time = time.time()
    metadata = MetaData()
    metadata.reflect(bind=engine)
    # print(f"Metadata reflection took {time.time() - start_time:.4f} seconds")

    start_time = time.time()
    temp_table = Table(table_name, metadata, autoload_with=engine)
    column_names = [column.name for column in temp_table.columns]
    # print(f"Table loading and column extraction took {time.time() - start_time:.4f} seconds")

    start_time = time.time()
    prompt = get_sql_prompt(failed_solutions, question, column_names, df, ai_client).replace("temp_table", table_name)
    # print(prompt)
    # print(f"get_sql_prompt took {time.time() - start_time:.4f} seconds")

    solutions = []
    errors = []

    for attempt in range(n_tries):
        import time
        model = select_model(attempt)
        start_time = time.time()
        if attempt == 0:
            sql_response = ai_client.generate_response(prompt, model=model)
        else:
            error_msg = "; ".join(errors)
            sql_response = ai_client.generate_response(f"Previous solution failed with error: {error_msg}. Write a correct fault-proof SQL SELECT query that solves this precise task. The task was: {question}\n" + prompt)

        

        start_time = time.time()
        matches = find_near_matches('CODE:', sql_response, max_l_dist=2)
        if not matches:
            raise ValueError("Expected 'CODE:' in the response but not found.")
        sql_response = sql_response[matches[0].end:sql_response.rfind("`")].replace("sql", "").replace("`", "").strip()
        
        with open("sql_few_shot_logs_a.txt", "a", encoding='utf-8') as f:
            f.write(f"question={question}\n")
            f.write("Column names: "+ " ".join(column_names) + "\n")
            f.write(sql_response + "\n")
        
        # print(f"SQL response processing took {time.time() - start_time:.4f} seconds")

        try:
            start_time = time.time()
            with engine.connect() as connection:
                result = connection.execute(text(sql_response))
                rows = result.fetchall()
            # print(f"SQL execution and fetching took {time.time() - start_time:.4f} seconds")

            if len(rows) > 1:
                solutions.append({
                    "code": sql_response,
                    "result": str([row[0] for row in rows]).replace('"', '').replace('`', '').replace("sql", ""),
                    "success": True
                })
            elif rows:
                solutions.append({
                    "code": sql_response,
                    "result": str(rows[0][0]),
                    "success": True
                })
            else:
                solutions.append({
                    "code": sql_response,
                    "result": "__NO_RESULT__",
                    "success": False
                })
        except Exception as e:
            errors.append(str(e))
            # print(str(e))
            solutions.append({
                "code": sql_response,
                "result": "__EXECUTION_FAILED__",
                "success": False
            })
   
    return solutions if any(sol["success"] for sol in solutions) else []


## Voting


In [36]:
def vote_on_solutions(solutions: List[Dict], question: str, dataset_name: str, is_sample, ai_client) -> str:
    """
    Use LLM to vote on the best solution among successful attempts.
    """
    if len(solutions) == 1:
        return solutions[0]["result"]

    # Generate voting prompt and get response from AI client
    voting_prompt = generate_voting_prompt(solutions, question, dataset_name, is_sample)
    response = ai_client.generate_response(voting_prompt, model="meta-llama/llama-3.3-70b-instruct").replace("return", "").strip()

    # Log the response to a file for debugging
    with open("voting_log.txt", "a", encoding='utf-8') as f:

        trimmed_voting_prompt = f'Given the following solutions and their results for the task: "{question}"\n\n' + \
            '\n'.join([f'Solution Number {i+1}:\nCode: {r["code"]} Answer: {str(r["result"])[:50]} (may be truncated)\n' for i, r in enumerate(solutions)])
        
        f.write(f"-------------\nVoting Prompt:\n{trimmed_voting_prompt}\n\nResponse:\n{response}\n-------------------------")

    # Extract the solution number from the response
    response = response.split("ANSWER:")[-1].strip()
    solution_number = None
    # Use regex to find the solution number in the response
    match = re.search(r'\b\d+\b', response)
    if match:
        solution_number = int(match.group())
    # Return the result of the selected solution if valid, otherwise return the first solution
    if solution_number is not None and 1 <= solution_number <= len(solutions):
        return solutions[solution_number - 1]["result"]
    return solutions[0]["result"]


---

## E2E solution

In [37]:
def e2e_response(question, dataset_text, ai_client) -> List:
    prompt = f'''
    Question: {question}
    Dataset: {dataset_text}

    Analyze the data. Provide your final answer to the question based on the data. 
    If the question assumes several answers, use a list. Your answer should be in the form of one of the following:
    1. Boolean (True/False)
    2. List (e.g., ['Tree', 'Stone'])
    3. Number (e.g., 5)
    4. String (e.g., 'Spanish')

    Give extensive reasoning and then fianlly provide the answer starting with string "Final Answer:" in one of the four formats presented above (Boolean, List, Number, String). Your response should then be finished.
    
    '''
    # with open("prompt_log.txt", "a", encoding='utf-8') as log_file:
    #     log_file.write(f"{prompt}\n\n----------------------------\n\n")

    code_response = ai_client.generate_response(prompt, model= "minimax/minimax-01").strip()
    matches = find_near_matches('Final Answer:', code_response, max_l_dist=2)
    if not matches:  # Check if matches is empty
        raise ValueError("Expected 'Final Answer:' in the response but not found.")
        
    last_index = matches[-1].end  # Use the last match
    code_response = code_response[last_index:].strip()
    # Remove all markdown symbols
    code_response = code_response.replace('**', '').replace('*', '').replace('`', '').replace('#', '').replace('_', '').strip().split()[0]

    # Ensure the response is encoded in UTF-8 to avoid encoding errors
    code_response = code_response.encode('utf-8', errors='ignore').decode('utf-8')

    return [{
        "code": "End-to-End model (no code visible)", 
        "result": code_response,
        "success": True
    }]


In [38]:
def dataset_to_text(dataset_name: str, question: str, ai_client, is_sample: bool, truncate_limit: int = 10000000) -> str:
    """
    Loads a dataset by name and converts the first 'truncate_limit' rows into a markdown representation.
    """

    # Load the dataset using the dataset_name
    prompt = f"Given the question: '{question}', which columns from the dataset are necessary to answer it? The dataset contains the following columns: {', '.join(load_table(dataset_name, is_sample).columns)}. Please provide extensive reasoning and only then the column names as a comma-separated list started with [ and ended with ]."
    # print(prompt)
    columns_text = ai_client.generate_response(prompt).strip()
    # print(f"Columns identified: {columns_text}")

    columns_list = [col.strip().strip("'\"`") for col in columns_text[columns_text.rfind('[')+1:columns_text.rfind(']')].split(',')]
    # print(f"Columns identified2: {columns_list}")
    
    dataset = load_table(dataset_name, is_sample)[columns_list]
    
    
    markdown_text = dataset.to_markdown()
    # Ensure the markdown text is encoded in UTF-8 to avoid encoding errors
    markdown_text = markdown_text.encode('utf-8', errors='ignore').decode('utf-8')
    # with open('dataset_markdown.txt', 'a', encoding='utf-8') as f:
    #     f.write(markdown_text)
    return markdown_text

---

## Question reformulation experiment

In [39]:
def reformulate_question(dataset_name: str, question: str, is_sample: bool, ai_client):
    """
    Analyzes potential ambiguities in the question and reformulates it to be more precise.
    Returns the reformulated question.
    """
    # Get dataset info
    df = load_table(dataset_name, is_sample)
    schema = ', '.join([f"{col} ({df[col].dtype})" for col in df.columns])
    sample_data = df.head(5).to_markdown()
    sample_data = sample_data.encode('utf-8', errors='ignore').decode('utf-8')

    # Ask about ambiguities
    ambiguities_prompt = f"""Given this dataset schema: {schema}
And this sample data (attention: it is just 5 first rows, not whole dataset):
{sample_data}

For this question: "{question}"

What are 3 potential severe ambiguities or unclear aspects that could prevent from answering this question correctly? Important: DO NOT PROVIDE YOUR OWN DEFINITIONS OR CONSTRAINTS, only use information from provided question and schema. ALWAYS CHECK IF RELEVANT COLUMN IS IN SCHEME AND INCLUDE IT INTO THE QUESTION. DONT ASK FOR UNIQUE WHERE IT IS NOT SPECIFIED EXPLICITLY."""
    
    ambiguities = ai_client.generate_response(ambiguities_prompt, model="meta-llama/llama-3.3-70b-instruct").strip()
    
    # Ask for reformulation
    reformulation_prompt = f""""
examples: 
Original request:
Are there any Pokémon with a total stat greater than 700?
Reformulated request:
(
"details": "1. The original question lacks column specificity. We need to reference the exact column names.
2. We should specify how Pokémon are uniquely identified in the dataset.
3. Need to be precise about the total stat column name.",
"result": Are there any unique Pokémon, identified by their `number`, in the dataset where the `total` value is greater than 700?
)

Original request:
How many posts are in Spanish?
Reformulated request:
(
"details": "1. The original question is vague about how Spanish language is identified.
2. Need to specify the exact column name and value to check.
3. Should clarify we want a count of matching posts.",
"result": What is the count of posts in the dataset where the 'lang' column value exactly matches 'es'?
)

Original request:
What is the average rating of the posts in Russia from 2015 to 2025 in Moscow?
Reformulated request:
(
"details": "1. The original question specifies a geographic location and time frame, which needs to be reflected in the dataset.
2. Need to clarify the exact column name for the rating.
3. Should mention if we want the average rating for all posts or a specific subset based on the provided criteria.",
"result": What is the average rating of posts in the dataset where the 'status' column value is 'published' and the 'date' column is between '2015-01-01' and '2025-12-31' in Moscow?
)

#### Task Description:
As the AI assistant, your task is to rewrite the NL entered by the user based on the given
database information and reflection.
This NL has some flaws and got bad generation in the downstream models, so you need to make this
NL as reliable as possible.
The rewritten NL should express more complete and accurate database information requirements
as far as possible. In order to do this task well, you need to follow these steps to think and
process step by step:
1. Please review the given reflection and DB information, and first check whether the NL contains
the corresponding key information and the corresponding flaws. If they exists, please modify,
supplement or rewrite it in the statement of NL by combining the reflection and DB.
2. Please rewrite the original NL based on the above process. On the premise of providing more
complete and more accurate database information, the structure of the rewritten NL should be similar
to the original statement as far as possible. All rewritten statements do not allow delimiters,
clauses, additional hints or explanations. DONT CONVERT IT INTO QUERY. DONT ADD UNIQUE WHERE IT IS NOT SPECIFIED EXPLICITLY. PREFER NAMES INSTEAD OF IDs when presenting the answer.
(
"details": <YOUR STEP-BY-STEP THINKING DETAILS>,
"result": <YOUR FINAL REWRITED NL>
)
### INPUT:
SCHEMA: # Fill the database content
{schema}
NL: # Fill the flaw NL
{question}
Possible Ambiguities: 
{ambiguities}

### OUTPUT:
     
    """
    try:
        reformulated_question = ai_client.generate_response(reformulation_prompt, model="meta-llama/llama-3.3-70b-instruct").strip()
        reformulated_question = reformulated_question[reformulated_question.rfind('esult:') + len('esult:'):].strip()
    except:
        reformulated_question = question
    return reformulated_question


### Function to incorporate all solutions

In [40]:
def process_response(dataset_name: str, question: str, is_sample: bool, ai_client):
    """
    Main processing function that orchestrates the solution attempts.
    """
    solutions = []

    

    try:
        solutions = get_python_solutions(question, dataset_name, is_sample, ai_client)
    except Exception as e:
        print(f"Error in get_python_solutions: {e}")
        solutions = []

    start_time = time.time()

    failed_solutions = [s for s in solutions if not s["success"]]

    sql_results = []
    try:
        sql_results = sql_fallback(dataset_name, question, is_sample, failed_solutions, ai_client)
    except Exception as e:
        print(f"Error during sql_fallback: {e}")

    print(f"Time for sql_fallback: {time.time() - start_time:.2f} seconds")

    if sql_results:
        solutions.extend(sql_results)

    e2e_results = []
    try:
        dataset_text = dataset_to_text(dataset_name, question, ai_client, is_sample) 
    except Exception as e:
        print(f"Error during dataset_to_text: {e}")
    try:
        e2e_results = e2e_response(question, dataset_text, ai_client)
    except Exception as e:
        print(f"Error during e2e_results: {e}")

    if len(e2e_results) > 0:
        solutions.extend(e2e_results)


    # print("All solutions:", solutions)

    successful_solutions = [s for s in solutions if s["success"]]
    # print("Successful:", successful_solutions)
    if len(successful_solutions) > 0:
        start_time = time.time()
        with open("2_test.txt", "a", encoding='utf-8') as f:
            f.write(f"---\n{question}\n{solutions}")
        final_result = vote_on_solutions(successful_solutions, question, dataset_name, is_sample, ai_client)
        print(f"Time for vote_on_solutions: {time.time() - start_time:.2f} seconds")

        if isinstance(final_result, str):
            final_result = final_result.replace('"', '').replace("'", '')
        elif isinstance(final_result, list):
            final_result = str(final_result[0] if len(final_result) == 1 else final_result)

        return final_result, solutions
    return "No solution", []

## Running pipeline

In [41]:
import os

def main():
    batch_size = 1
    # Check how many lines are already in the file
    if os.path.exists("solution_results.txt"):
        with open("solution_results.txt", "r", encoding="utf-8") as f:
            processed_count = sum(1 for _ in f)
    else:
        processed_count = 0
    print(f"Starting to fill the file from line N {processed_count+1}")
    ai_client=OpenAIClient()
    dataset = load_data()
    start_index = processed_count
    end_index = min(start_index + batch_size, len(dataset))
    dataset = dataset.select(range(start_index, end_index))
    

    responses = []
    # Process the current batch and append responses one by one
    with open("solution_results.txt", "a", encoding="utf-8") as f:
        for row in tqdm(dataset):
            try:
                response, log_info = process_response(row["dataset"], row["question"], is_sample=False, ai_client=ai_client)
                f.write(f"{str(response).replace(chr(10), ' ')}\n")
                responses.append(response)
            except Exception as e:
                print(f"An error occurred on row {row}: {e}")
                break

    accuracy, *meta_info = evaluate_responses(responses, dataset, is_sample=False)
    print(f"DataBench accuracy: {accuracy}")

    with open("solution_results_logs.txt", "a", encoding="utf-8") as log_file:
        for sublist in meta_info:
            for response, truth, semantic, question, verdict in sublist:
                log_file.write(f"{response=} | {truth=} | {semantic=} | {question=} | {accuracy=} | {verdict=}\n")


In [ ]:
for batch_size in range(100):
    main()